In [ ]:
# # install google chrome
# !wget https://dl.google.com/linux/linux_signing_key.pub
# !sudo apt-key add linux_signing_key.pub
# !echo 'deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main' >> /etc/apt/sources.list.d/google-chrome.list
# !sudo apt-get -y update
# !sudo apt-get install -y google-chrome-stable

# # install chromedriver
# # !apt-get install -y qq unzip
# !wget -O /tmp/chromedriver.zip http://chromedriver.storage.googleapis.com/`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`/chromedriver_linux64.zip
# !unzip /tmp/chromedriver.zip chromedriver -d /usr/bin/

In [ ]:
# # To check Google Chrome's version
# !google-chrome --version

# # To check Chrome Driver's version
# !chromedriver -v

In [ ]:
# # Install Firefox
# !sudo install -d -m 0755 /etc/apt/keyrings
# !wget -q https://packages.mozilla.org/apt/repo-signing-key.gpg -O- | sudo tee /etc/apt/keyrings/packages.mozilla.org.asc > /dev/null
# !gpg -n -q --import --import-options import-show /etc/apt/keyrings/packages.mozilla.org.asc | awk '/pub/{getline; gsub(/^ +| +$/,""); if($0 == "35BAA0B33E9EB396F59CA838C0BA5CE6DC6315A3") print "\nThe key fingerprint matches ("$0").\n"; else print "\nVerification failed: the fingerprint ("$0") does not match the expected one.\n"}'
# !echo "deb [signed-by=/etc/apt/keyrings/packages.mozilla.org.asc] https://packages.mozilla.org/apt mozilla main" | sudo tee -a /etc/apt/sources.list.d/mozilla.list > /dev/null
# !echo 'Package: * Pin: origin packages.mozilla.org Pin-Priority: 1000' | sudo tee /etc/apt/preferences.d/mozilla
# !sudo apt-get update && sudo apt-get install firefox -y

In [ ]:
# Install Microsoft Edge
!curl https://packages.microsoft.com/keys/microsoft.asc | gpg --dearmor > microsoft.gpg
!sudo install -o root -g root -m 644 microsoft.gpg /etc/apt/trusted.gpg.d/
!sudo sh -c 'echo "deb [arch=amd64] https://packages.microsoft.com/repos/edge stable main" > /etc/apt/sources.list.d/microsoft-edge-dev.list'
!sudo rm microsoft.gpg
!sudo apt update && sudo apt install microsoft-edge-stable -y

In [ ]:
!sudo apt install -y python3-selenium

In [ ]:
!pip install biopython
!pip install selenium
!pip install webdriver-manager

In [ ]:
# !pip install tqdm
# !pip install urllib3

In [ ]:
# from selenium import webdriver
# from selenium.webdriver.edge.options import Options
# from selenium.webdriver.edge.service import Service
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support import expected_conditions as EC
# # from selenium.webdriver.support.wait import WebDriverWait
# from selenium.webdriver.support.ui import WebDriverWait
# # from webdriver_manager.microsoft import GeckoDriverManager
# from webdriver_manager.microsoft import EdgeChromiumDriverManager

# from tqdm import tqdm
# import logging
# import urllib.request as req
# import os
# import numpy as np

# def crawl():
#     # Read fasta file:
#     prots = read_fasta(SP6_PATH)

#     # Open driver
#     options = Options()
# #     options.headless = True
# #     options = webdriver.FirefoxOptions()
#     options.add_argument('--headless')
# #     options.add_argument('--disable-gpu')
#     options.add_argument('--no-sandbox')
#     options.add_argument('--disable-dev-shm-usage')
# #     chrome_options.add_argument("--window-size=1920,1080")
#     driver = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()), options=options)

#     error_ids = []

#     # for loop to get file
#     for _, prot_id in tqdm(enumerate(prots)):
#         try:
#             # access Uniprot page
#             driver.get(f'https://www.uniprot.org/uniprotkb/{prot_id}/entry#structure')
#             alphafold_url = WebDriverWait(driver, 30).until(
#                 EC.presence_of_element_located((By.XPATH, "//a[text()='AlphaFold']"))
#             ).get_attribute('href')

#             # redirect to AlphaFold page
#             driver.get(alphafold_url)
#             download_url = WebDriverWait(driver, 30).until(
#                 EC.presence_of_element_located((By.XPATH, "//a[text()='PDB file ']"))
#             ).get_attribute('href')

#             if not os.path.exists(SAVE_DIR):
#                 os.makedirs(SAVE_DIR, exist_ok=True)

#             filename = SAVE_DIR + '/' + download_url.split('/')[-1]
#             req.urlretrieve(download_url, filename)

#         except Exception as e:
#             error_ids.append(prot_id)
#             logging.exception(e)

#     np.savetxt(KAGGLE_DIR + '/error_ids.txt', error_ids, fmt="%s")
#     driver.close()

In [ ]:
import logging


class Logger(object):
    def __init__(self, filename=None, use_console: bool = False):
        # reset logging to default
        logging.root.handlers = []
        self.log_dir = WORKING_DIR
        self.filename = filename
        self.use_console = use_console
        self.logger = logging.getLogger(__name__)
        self._configure_logging()

    def _configure_logging(self):
        formatter = logging.Formatter(
            fmt='%(asctime)s (%(name)s:%(lineno)d) [%(levelname)s]: %(message)s',
            datefmt='%m/%d/%Y %I:%M:%S %p'
        )

        file_handler = logging.FileHandler(self.filename, encoding='utf-8', delay=False)
        file_handler.setFormatter(formatter)
        self.logger.addHandler(file_handler)

        if self.use_console:
            stream_handler = logging.StreamHandler()
            stream_handler.setFormatter(formatter)
            self.logger.addHandler(stream_handler)

        self.logger.setLevel(logging.DEBUG)

    def info(self, msg):
        return self.logger.info(msg)

    def warning(self, msg):
        return self.logger.warning(msg)

    def error(self, msg):
        return self.logger.error(msg)


In [ ]:
import json
import time
# import time
import urllib.request as req
import os

import numpy as np
from Bio import SeqIO
from selenium import webdriver
from selenium.webdriver.edge.options import Options
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from webdriver_manager.microsoft import EdgeChromiumDriverManager

# UNIPROT_URL = 'https://www.uniprot.org/uniprot'
WORKING_DIR = '/kaggle/working'
INPUT_DIR = '/kaggle/input/crawler-cache'
SP6_PATH = f'{INPUT_DIR}/train_set.fasta'
SAVE_DIR = f'{WORKING_DIR}/pdb'

# options = webdriver.ChromeOptions()
# driver = webdriver.Chrome(options=options)

logger = Logger(filename=f'{WORKING_DIR}/crawler.log', use_console=True)


def extract_prot_ids_fasta(fasta_file=SP6_PATH):
    prot_ids = []
    records = SeqIO.parse(fasta_file, 'fasta')
    for record in records:
        annotation = str(record.id).split('|')
        prot_ids.append(annotation[0])
    return prot_ids


def load_cache():
    done_ids = np.loadtxt(f'{INPUT_DIR}/done_ids.txt', dtype=str)
    missing_ids = np.loadtxt(f'{INPUT_DIR}/missing_ids.txt', dtype=str)
    return done_ids.tolist(), missing_ids.tolist()


def save_cache(done_ids, missing_ids):
    np.savetxt(fname=f'{WORKING_DIR}/done_ids.txt', X=done_ids, fmt='%s')
    np.savetxt(fname=f'{WORKING_DIR}/missing_ids.txt', X=missing_ids, fmt='%s')


# First try to download
def download_pdb_by_alphafold():
    prot_ids = extract_prot_ids_fasta()
    if not os.path.exists(SAVE_DIR):
        os.makedirs(SAVE_DIR, exist_ok=True)
    done_ids, missing_ids = load_cache()
    for i, prot_id in enumerate(prot_ids):
        if i > 0 and i % 100 == 0:
            logger.info(f'Processing {i}/{len(prot_ids)} records')
        if prot_id not in done_ids and prot_id not in missing_ids:
            download_url = f'https://alphafold.ebi.ac.uk/files/AF-{prot_id}-F1-model_v4.pdb'
            filename = SAVE_DIR + f'/AF-{prot_id}-F1-model_v4.pdb'
            try:
                req.urlretrieve(download_url, filename)
                logger.info(f'Downloaded at {filename}')
                done_ids.append(prot_id)
            except Exception as e:
                logger.error(f'Failed to download {prot_id}: {e}')
                missing_ids.append(prot_id)
            if i > 0 and i % 100 == 0:
                # logger.info(f'Processing thought {i} records')
                save_cache(done_ids, missing_ids)

        # time.sleep(3)  # prevent bot detection (do not necessary)

    # save after running
    save_cache(done_ids, missing_ids)


# Retry
def retry_download_from_uniprot():
    convert_ids = {}
    done_ids, missing_ids = load_cache()
    options = Options()
    options.add_argument('--headless')
    # options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    # options.add_argument("--window-size=1920,1080")

    driver = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()), options=options)
    for i, prot_id in enumerate(missing_ids):
        driver.get(f'https://www.uniprot.org/uniprotkb/{prot_id}/entry#structure')
        time.sleep(3)
        try:
            # Retry downloading ids missing from downloading process above by accessing from uniprot page
            current_url = driver.current_url
            af_url = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//a[text()='AlphaFold"))
            ).get_attribute('href')
            new_prot_id = current_url.split('/')[-2]  # retrieve new prot_id
            driver.get(af_url)
            time.sleep(3)
            download_url = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//a[text()='PDB file ']"))
            ).get_attribute('href')
            filename = f'{SAVE_DIR}/AF-{new_prot_id}-F1-model_v4.pdb'
            try:
                req.urlretrieve(download_url, filename)
                missing_ids = missing_ids.remove(prot_id)
                done_ids = done_ids.append(prot_id)
                if prot_id not in convert_ids.keys():
                    convert_ids[prot_id] = new_prot_id
                time.sleep(3)
            except Exception as e:
                logger.error(f'Failed to download {prot_id}: {e}')
        except Exception:
            logger.error(f'No AlphaFoldl url found')

        if i > 0 and i % 100 == 0:  # save cache after every 100 epochs
            save_cache(done_ids, missing_ids)

    # Save convert ids of success retrying and update cache
    save_cache(done_ids, missing_ids)
    with open(f'{WORKING_DIR}/convert_ids.json', 'w') as f:
        json.dump(convert_ids, f)

    # close driver
    driver.close()


def crawl():
    download_pdb_by_alphafold()
    retry_download_from_uniprot()

In [ ]:
crawl()

In [ ]:
# !pip list